In [ ]:
!git clone https://github.com/ndb796/weather_dataset
%cd weather_dataset

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers


import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#dataset loader setup

train_generator = ImageDataGenerator(
    rescale=1/255.,
    horizontal_flip=True,
    validation_split = 0.2
)

test_generator = ImageDataGenerator(
    rescale=1/255.
)

In [ ]:
train_flow = train_generator.flow_from_directory(
    directory='train/',
    shuffle=True,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 64,
    subset='training'
)

val_flow = train_generator.flow_from_directory(
    directory='train/',
    shuffle=True,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 64,
    subset='validation'
)

test_flow = test_generator.flow_from_directory(
    directory='test/',
    shuffle = False,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size = 64
)

In [ ]:
class_names = { 
    0: "흐림", 
    1: "비",
    2: "맑음",
    3: "일출" 
}

In [ ]:
images,targets = next(train_flow)



In [ ]:
for i in range(5):
    print(class_names[np.argmax(targets[i])])
    plt.imshow( images[i] )
    plt.show()

In [ ]:
## simpel model 
model  = Sequential()

model.add( layers.InputLayer(shape=(224, 224, 3))) 
model.add( layers.Flatten() )
model.add( layers.Dense(4, activation='softmax')) 



In [ ]:
## deeper model

model2 = Sequential()

model2.add( layers.InputLayer(shape=(224, 224, 3)))
model2.add( layers.Flatten() ) 
model2.add( layers.Dense(64, activation='relu'))
model2.add( layers.Dense(4, activation='softmax'))



In [ ]:
## more deeper model

model3 = Sequential()

model3.add(layers.InputLayer(shape=(244, 244, 3)))
model2.add( layers.Flatten() ) 
model2.add( layers.Dense(128, activation='relu'))
model2.add( layers.Dense(64, activation='relu'))
model2.add( layers.Dense(32, activation='relu'))
model2.add( layers.Dense(4, activation='softmax'))




In [ ]:
model.compile( optimizer= 'sgd', loss = 'crossentropy', metrics=['accuracy'] )
history = model.fit(train_flow, epochs=30, validation_data=val_flow)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
model2.compile( optimizer= 'sgd', loss = 'crossentropy', metrics=['accuracy'] )
history2 = model2.fit(train_flow, epochs=30, validation_data=val_flow)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])

In [ ]:
model3.compile( optimizer= 'sgd', loss = 'crossentropy', metrics=['accuracy'] )
history3 = model3.fit(train_flow, epochs=30, validation_data=val_flow)

In [ ]:
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])